In [ ]:
train_data = {
  'good': True,
  'bad': False,
  'happy': True,
  'sad': False,
  'not good': False,
  'not bad': True,
  'not happy': False,
  'not sad': True,
  'very good': True,
  'very bad': False,
  'very happy': True,
  'very sad': False,
  'i am happy': True,
  'this is good': True,
  'i am bad': False,
  'this is bad': False,
  'i am sad': False,
  'this is sad': False,
  'i am not happy': False,
  'this is not good': False,
  'i am not bad': True,
  'this is not sad': True,
  'i am very happy': True,
  'this is very good': True,
  'i am very bad': False,
  'this is very sad': False,
  'this is very happy': True,
  'i am good not bad': True,
  'this is good not bad': True,
  'i am bad not good': False,
  'i am good and happy': True,
  'this is not good and not happy': False,
  'i am not at all good': False,
  'i am not at all bad': True,
  'i am not at all happy': False,
  'this is not at all sad': True,
  'this is not at all happy': False,
  'i am good right now': True,
  'i am bad right now': False,
  'this is bad right now': False,
  'i am sad right now': False,
  'i was good earlier': True,
  'i was happy earlier': True,
  'i was bad earlier': False,
  'i was sad earlier': False,
  'i am very bad right now': False,
  'this is very good right now': True,
  'this is very sad right now': False,
  'this was bad earlier': False,
  'this was very good earlier': True,
  'this was very bad earlier': False,
  'this was very happy earlier': True,
  'this was very sad earlier': False,
  'i was good and not bad earlier': True,
  'i was not good and not happy earlier': False,
  'i am not at all bad or sad right now': True,
  'i am not at all good or happy right now': False,
  'this was not happy and not good earlier': False,
}

test_data = {
  'this is happy': True,
  'i am good': True,
  'this is not happy': False,
  'i am not good': False,
  'this is not bad': True,
  'i am not sad': True,
  'i am very good': True,
  'this is very bad': False,
  'i am very sad': False,
  'this is bad not good': False,
  'this is good and happy': True,
  'i am not good and not happy': False,
  'i am not at all sad': True,
  'this is not at all good': False,
  'this is not at all bad': True,
  'this is good right now': True,
  'this is sad right now': False,
  'this is very bad right now': False,
  'this was good earlier': True,
  'i was not happy and not good earlier': False,
}


In [ ]:
class RNN:
  # A Vanilla Recurrent Neural Network.

  def __init__(self, input_size, output_size, hidden_size=64):
    # Weights
    self.Whh = randn(hidden_size, hidden_size) / 1000
    self.Wxh = randn(hidden_size, input_size) / 1000
    self.Why = randn(output_size, hidden_size) / 1000

    # Biases
    self.bh = np.zeros((hidden_size, 1))
    self.by = np.zeros((output_size, 1))

  def softmax(self, xs): # 算每个类的概率
    return np.exp(xs) / sum(np.exp(xs))

  def forward(self, inputs): # 前向传播
    hs = dict()
    hs[-1] = np.zeros((self.Whh.shape[0], 1))
    for i, x in enumerate(inputs):
      hs[i] = np.tanh(self.Wxh @ x + self.Whh @ hs[i-1] + self.bh)  
    self.hs = hs

    y = self.Why @ hs[len(inputs)-1] + self.by
    self.p = self.softmax(y)

    self.inputs = inputs

    return self.p

  def backprop(self, label, learn_rate = 2e-2): # 反向传播
    d_y = self.p
    d_y[label] -= 1

    d_Why = d_y @ self.hs[len(self.inputs)-1].T
    d_by = d_y

    d_h = self.Why.T @ d_y
    # Initialize dL/dWhh, dL/dWxh, and dL/dbh to zero.
    d_Whh = np.zeros(self.Whh.shape)
    d_Wxh = np.zeros(self.Wxh.shape)
    d_bh = np.zeros(self.bh.shape)
    for i in reversed(range(len(self.inputs))):
      temp = (1 - self.hs[i]**2) * d_h # temp = (1-hj^2) * d_L_d_h

      d_bh += temp
      d_Wxh += temp @ self.inputs[i].T
      d_Whh += temp @ self.hs[i-1].T

      d_h = self.Whh @ temp

    # Clip to prevent exploding gradients.
    for d in [d_Wxh, d_Whh, d_Why, d_bh, d_by]:
      np.clip(d, -1, 1, out=d) # 截取函数，限制在-1到1之间

    self.Whh -= learn_rate * d_Whh
    self.Wxh -= learn_rate * d_Wxh
    self.Why -= learn_rate * d_Why
    self.bh -= learn_rate * d_bh
    self.by -= learn_rate * d_by

In [ ]:
import numpy as np
from numpy.random import randn
import random

In [ ]:
# Initialize our RNN!
rnn = RNN(vocab_size, 2)

In [ ]:
# Create the vocabulary.
vocab = list(set([w for text in train_data.keys() for w in text.split(' ')]))
vocab_size = len(vocab)
word_to_idx = { w: i for i, w in enumerate(vocab) }
idx_to_word = { i: w for i, w in enumerate(vocab) }

def createInputs(text): # 将一句话里的单词们转为one-hot向量
  inputs = []
  for w in text.split(' '):
    v = np.zeros((vocab_size, 1))
    v[word_to_idx[w]] = 1
    inputs.append(v)
  return inputs

In [ ]:
def processData(data, backprop=True):
  items = list(data.items())
  random.shuffle(items)

  loss = 0
  num_correct = 0

  for x, y in items:
    inputs = createInputs(x) # 把句子x转为向量inputs
    target = int(y) # 句子x对应的类别

    # Forward
    probs = rnn.forward(inputs)

    # Calculate loss / accuracy
    loss -= np.log(probs[target])
    num_correct += int(np.argmax(probs) == target)

    if backprop:
      # Backward
      rnn.backprop(target)

  return loss / len(data), num_correct / len(data)

In [ ]:
# Training loop
for epoch in range(1000):
  train_loss, train_acc = processData(train_data)

  if epoch % 100 == 99:
    print('--- Epoch %d' % (epoch + 1))
    print('Train:\tLoss %.3f | Accuracy: %.3f' % (train_loss, train_acc))

    test_loss, test_acc = processData(test_data, backprop=False)
    print('Test:\tLoss %.3f | Accuracy: %.3f' % (test_loss, test_acc))

--- Epoch 100
Train:	Loss 0.688 | Accuracy: 0.552
Test:	Loss 0.696 | Accuracy: 0.500
--- Epoch 200
Train:	Loss 0.667 | Accuracy: 0.690
Test:	Loss 0.717 | Accuracy: 0.450
--- Epoch 300
Train:	Loss 0.599 | Accuracy: 0.672
Test:	Loss 0.690 | Accuracy: 0.500
--- Epoch 400
Train:	Loss 0.402 | Accuracy: 0.914
Test:	Loss 0.639 | Accuracy: 0.650
--- Epoch 500
Train:	Loss 0.172 | Accuracy: 0.931
Test:	Loss 0.387 | Accuracy: 0.800
--- Epoch 600
Train:	Loss 0.008 | Accuracy: 1.000
Test:	Loss 0.129 | Accuracy: 0.950
--- Epoch 700
Train:	Loss 0.004 | Accuracy: 1.000
Test:	Loss 0.061 | Accuracy: 1.000
--- Epoch 800
Train:	Loss 0.002 | Accuracy: 1.000
Test:	Loss 0.019 | Accuracy: 1.000
--- Epoch 900
Train:	Loss 0.001 | Accuracy: 1.000
Test:	Loss 0.008 | Accuracy: 1.000
--- Epoch 1000
Train:	Loss 0.001 | Accuracy: 1.000
Test:	Loss 0.005 | Accuracy: 1.000
